In [1]:
import numpy as np

In [2]:
def play_game(p1, p2, env, draw=False):
    current_player = None
    
    # loops until the game is over
    while not env.game_over():
        # alternate between players (p1 always first)
        if current_player == p1:
            current_player = p2
        else:
            current_player = p1
        
        # make a move
        current_player.take_action(env)
        
        # draw if necessary
        if draw:
            env.draw_board()
        
        # update histories
        state = env.get_state()
        p1.update_state_history(state)
        p2.update_state_history(state)
    
    # value function update
    p1.update(env)
    p2.update(env)
